In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

if os.environ['OPENAI_API_KEY']:
    print("API Key is set.")

API Key is set.


In [4]:
from langchain_openai import ChatOpenAI


In [5]:
llm = ChatOpenAI(model="gpt-5-nano",temperature=0)

## **RAG IMPLEMENTATION WITH YOUR OWN TEXT DATA**

#### **STEP 1: Preparing Document for your Text**

In [10]:
from langchain_core.documents import Document

## Your text data
my_text = """Artificial intelligence (AI) is the capability of computational systems to perform tasks typically associated with human intelligence, such as learning, reasoning, problem-solving, perception, and decision-making. It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and use learning and intelligence to take actions that maximize their chances of achieving defined goals.[1]

High-profile applications of AI include advanced web search engines (e.g., Google Search); recommendation systems (used by YouTube, Amazon, and Netflix); virtual assistants (e.g., Google Assistant, Siri, and Alexa); autonomous vehicles (e.g., Waymo); generative and creative tools (e.g., language models and AI art); and superhuman play and analysis in strategy games (e.g., chess and Go). However, many AI applications are not perceived as AI: "A lot of cutting edge AI has filtered into general applications, often without being called AI because once something becomes useful enough and common enough it's not labeled AI anymore."[2][3]

Various subfields of AI research are centered around particular goals and the use of particular tools. The traditional goals of AI research include learning, reasoning, knowledge representation, planning, natural language processing, perception, and support for robotics.[a] To reach these goals, AI researchers have adapted and integrated a wide range of techniques, including search and mathematical optimization, formal logic, artificial neural networks, and methods based on statistics, operations research, and economics.[b] AI also draws upon psychology, linguistics, philosophy, neuroscience, and other fields.[4] Some companies, such as OpenAI, Google DeepMind and Meta,[5] aim to create artificial general intelligence (AGI) – AI that can complete virtually any cognitive task at least as well as a human.
""" 


docs = [Document(page_content=my_text,metadata={"source":"ABC","documentID":"Doc1"})]
docs


[Document(metadata={'source': 'ABC', 'documentID': 'Doc1'}, page_content='Artificial intelligence (AI) is the capability of computational systems to perform tasks typically associated with human intelligence, such as learning, reasoning, problem-solving, perception, and decision-making. It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and use learning and intelligence to take actions that maximize their chances of achieving defined goals.[1]\n\nHigh-profile applications of AI include advanced web search engines (e.g., Google Search); recommendation systems (used by YouTube, Amazon, and Netflix); virtual assistants (e.g., Google Assistant, Siri, and Alexa); autonomous vehicles (e.g., Waymo); generative and creative tools (e.g., language models and AI art); and superhuman play and analysis in strategy games (e.g., chess and Go). However, many AI applications are not perceived as AI: "A lot of c

#### **STEP 2: Splitting the Document into CHUNKS**

In [11]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50)

chunks = splitter.split_documents(docs)
chunks

[Document(metadata={'source': 'ABC', 'documentID': 'Doc1'}, page_content='Artificial intelligence (AI) is the capability of computational systems to perform tasks typically associated with human intelligence, such as learning, reasoning, problem-solving, perception, and decision-making. It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and use learning and intelligence to take actions that maximize their chances of achieving defined goals.[1]'),
 Document(metadata={'source': 'ABC', 'documentID': 'Doc1'}, page_content='High-profile applications of AI include advanced web search engines (e.g., Google Search); recommendation systems (used by YouTube, Amazon, and Netflix); virtual assistants (e.g., Google Assistant, Siri, and Alexa); autonomous vehicles (e.g., Waymo); generative and creative tools (e.g., language models and AI art); and superhuman play and analysis in strategy games (e.g., chess a

#### **STEP 3: Creating Embeddings for the Chunks**

In [ ]:
from langchain_openai import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

#### **STEP 4: Create and Store Embeddings in Vector Store**

In [17]:
from langchain_community.vectorstores import Chroma

vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embedding_model
)

#### **STEP 5: Semantic Search**

In [18]:
vectorstore.similarity_search("What is AI?", k= 3)

[Document(metadata={'documentID': 'Doc1', 'source': 'ABC'}, page_content='Artificial intelligence (AI) is the capability of computational systems to perform tasks typically associated with human intelligence, such as learning, reasoning, problem-solving, perception, and decision-making. It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and use learning and intelligence to take actions that maximize their chances of achieving defined goals.[1]'),
 Document(metadata={'source': 'ABC', 'documentID': 'Doc1'}, page_content='High-profile applications of AI include advanced web search engines (e.g., Google Search); recommendation systems (used by YouTube, Amazon, and Netflix); virtual assistants (e.g., Google Assistant, Siri, and Alexa); autonomous vehicles (e.g., Waymo); generative and creative tools (e.g., language models and AI art); and superhuman play and analysis in strategy games (e.g., chess a

#### **Talk to LLM**

In [19]:
context = vectorstore.similarity_search("What is AI?", k= 3)

In [22]:
response = llm.invoke(f"What is AI? You can answer using the following context: {context}")
print(response.content)

AI, or artificial intelligence, is the capability of computational systems to perform tasks that are typically associated with human intelligence. This includes learning, reasoning, problem-solving, perception, and decision-making. It’s a field of computer science that studies and develops methods and software that let machines perceive their environment and use learning and intelligence to take actions aimed at achieving defined goals.

Key points:
- AI spans technologies like neural networks, statistics, operations research, and economics, and it draws on insights from psychology, linguistics, philosophy, neuroscience, and more.
- Some researchers aim for artificial general intelligence (AGI): AI that can perform virtually any cognitive task as well as a human.

Common applications:
- Web search engines (e.g., Google Search)
- Recommendation systems (YouTube, Amazon, Netflix)
- Virtual assistants (Siri, Google Assistant, Alexa)
- Autonomous vehicles (Waymo)
- Generative and creative 